<a href="https://colab.research.google.com/github/kim-juhyeon/Practical-Time-Series-Analysis/blob/master/Ch/2%EC%9E%A5_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
emails = pd.read_csv('/content/drive/MyDrive/BookRepo/Ch02/data/emails.csv')
donations = pd.read_csv("/content/drive/MyDrive/BookRepo/Ch02/data/donations.csv")
YearJoined = pd.read_csv("/content/drive/MyDrive/BookRepo/Ch02/data/year_joined.csv")
donations

,amount,timestamp,user
0,25.0,2017-11-12 11:13:44,0.0
1,50.0,2015-08-25 19:01:45,0.0
2,25.0,2015-03-26 12:03:47,0.0
3,50.0,2016-07-06 12:24:55,0.0
4,50.0,2016-05-11 18:13:04,1.0
...,...,...,...
2671,25.0,2016-09-02 11:20:00,992.0
2672,50.0,2017-11-02 12:17:06,993.0
2673,1000.0,2016-09-13 21:09:47,995.0
2674,1000.0,2017-09-29 20:03:01,995.0


### 이메일과 기부 데이터를 어떻게 서로 연관 지어 볼까?
- 기부 데이터를 다운샘플링하여 주간 시계열로 바꿀 수 있다.
- 사실상 주간 기부금은 기부자 개별의 기부 금액이 반영된 경우가 대부분입니다.(한 주에 두 번 이상 기분하는 사람이 있을 가능성이 낮기 때문이다.)

In [ ]:
donations.timestamp = pd.to_datetime(donations.timestamp) # 팬더스에 내장된 날짜 관련 색인 기능을 사용하여 문자열을 타임스탬프에 적절한 데이터로 변환합니다.
donations.set_index('timestamp', inplace=True) # set_index는 기존 인덱스를 대체하거나 확장할 수 있습니다. inplace는 True이면 DataFrame을 제자리에서 수정합니다(새 개체를 만들지 않음).
agg_don = donations.groupby('user').apply(lambda df: df.amount.resample("W-MON").sum().dropna())# 주를 월요일로 고정한 다음 재배열합니다. 그러면 이메일 데이터의 주간 날짜와 일치합니다.
# 기부금 총액을 구하고, 기부가 없는 주는 삭제하는 작업을 진행
agg_don.head

<bound method NDFrame.head of user   timestamp 
0.0    2015-03-30      25.0
       2015-04-06       0.0
       2015-04-13       0.0
       2015-04-20       0.0
       2015-04-27       0.0
                      ...  
995.0  2017-09-11       0.0
       2017-09-18       0.0
       2017-09-25       0.0
       2017-10-02    1000.0
998.0  2018-01-08      50.0
Name: amount, Length: 32352, dtype: float64>

In [ ]:
#2장_01

complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.user)))
all_email = emails.set_index(['week', 'user']).reindex(complete_idx, fill_value=0).reset_index()
all_email.columns = ['week', 'user', 'emailsOpened']
all_email[all_email.user == 997].sort_values('week') #오름차순 (sort)
cutoff_dates = emails.groupby('user').week.agg(['min', 'max']).reset_index()
cutoff_dates = cutoff_dates.reset_index()
import warnings
warnings.filterwarnings('ignore')

for _, row in cutoff_dates.iterrows():
  user        = row['user']
  start_date  = row['min']
  end_date    = row['max']
  all_email.drop(all_email[all_email.user == user][all_email.week < start_date].index, inplace=True)
  all_email.drop(all_email[all_email.user == user][all_email.week > end_date].index, inplace=True)

----------------

In [ ]:
## 회원별 이메일 및 기부에 대한 데이터프레임 통합!
merged_df = pd.DataFrame()

for user, user_email in all_email.groupby('user'):
  ##특정 회원의 기부 데이터를 추출합니다.
  user_donations = agg_don[agg_don.index.get_level_values('user') == user] # agg_don user와 email user를 통합
  
  user_donations = user_donations.droplevel(0) # DataFrame을 유지하면서 0번째 index(열)삭제
  # user_donations.set_index('timestamp', inplace = True)  
  
 

In [ ]:
  user_email = all_email[all_email.user == user]
  user_email.sort_values('week', inplace=True)
  user_email.set_index('week', inplace=True)

  df = pd.merge(user_email, user_donations, how='left', left_index=True, right_index=True) # user_email,user_donations 은 유저이메일 기준으로 기부금을 통합한다. 그러므로 기부금값에 amount값은 nan이 나올 수 있음 
  # 왜냐? email user값 기준으로 머지 했기 때문
  df.fillna(0) # nan값을 "0"으로 채움

  merged_df = merged_df.append(df.reset_index()[['user', 'week', 'emailsOpened', 'amount']])


In [ ]:
merged_df # 이메일과 기부 데이터가 회원별로 줄지어 정렬된다. 각 회원 대해 의미 있는 주만 포함하고 각 회원의 멤버십기간 전과 후에 대한 주는 포함하지 않습니다.

,user,week,emailsOpened,amount
0,999.0,2015-06-08 00:00:00,2.0,NaN
1,999.0,2015-06-15 00:00:00,1.0,NaN
2,999.0,2015-06-22 00:00:00,0.0,NaN
3,999.0,2015-06-29 00:00:00,2.0,NaN
4,999.0,2015-07-06 00:00:00,3.0,NaN
...,...,...,...,...
150,999.0,2018-04-23 00:00:00,3.0,NaN
151,999.0,2018-04-30 00:00:00,3.0,NaN
152,999.0,2018-05-07 00:00:00,3.0,NaN
153,999.0,2018-05-14 00:00:00,3.0,NaN


In [ ]:
df = merged_df[merged_df.user == 999]
df['target'] = df.amount.shift(1) #df의 amount 열의 기록을 1만큼 뒤로 미룹니다.
df = df.fillna(0)
df.head(7)
# 이메일 응답 방식을 사용해서 회원의 다음 기부를 예측하는 모델을 만든다고 가정해보자. 이때 시간에 따른 이메일 열람의 행동 패턴을 하나의 지표로 고려합니다. 그러면 한 주의 기부금 
# 기부가 발생하기 전 주의 이메일 응답 방식에 따라서 정렬해야 합니다.

,user,week,emailsOpened,amount,target
0,999.0,2015-06-08 00:00:00,2.0,0.0,0.0
1,999.0,2015-06-15 00:00:00,1.0,0.0,0.0
2,999.0,2015-06-22 00:00:00,0.0,0.0,0.0
3,999.0,2015-06-29 00:00:00,2.0,0.0,0.0
4,999.0,2015-07-06 00:00:00,3.0,0.0,0.0
5,999.0,2015-07-13 00:00:00,3.0,0.0,0.0
6,999.0,2015-07-20 00:00:00,3.0,0.0,0.0


### 의미 있는 시간 규모란
계통 오차를 줄이기 위해서는 판매 데이터의 시간을 일 단위에서 주 단위로 바꾸는 것을 고려해볼 수 도 있습니다. </br>그렇지 않으면 요일에 띠라 오차가 편향될 가능성을 고려한 
모델을 만들어야 하기 때문<br>
